Tutorial here: https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html

#### Using CLI Credentials 
Flywheel CLI: https://docs.flywheel.io/hc/en-us/articles/360008162214
If you’ve logged in using the CLI, you can create a client instance without using an API key. This is useful when sharing SDK scripts for others to use.

#### Generate API key
Go to TMII Flywheel: https://tmii-flywheel.mssm.edu/  
  * Navigate to your Profile > Generate API Key

Your API key can be used to make HTTP requests to the API by including your API key in an authorization header. The custom line below, which includes your key, provides an example of how you can use `curl` to get a list of projects and related information.

```
curl -H "Authorization: scitran-user tmii-flywheel.mssm.edu:APIKEY" https://tmii-flywheel.mssm.edu/api/projects
```

In [1]:
# WARNING:Flywheel:Use "pip install flywheel-sdk==14.6.2" to install a compatible version for this server
# import flywheel package
import flywheel

# Create client, using CLI credentials

fw = flywheel.Client()
self = fw.get_current_user()
print('I am %s %s' % (self.firstname, self.lastname))


I am Tien Tong


In [2]:
project = fw.projects.find_one('label=K23')

In [ ]:
%%bash

fw export bids --project K23 ~/Documents/test


# Create a de-id profile

https://docs.flywheel.io/hc/en-us/articles/1500003050521-Create-a-de-id-profile

Step 2: Generate a de-id template
To begin, we will generate a de-id template. In later steps we will update the template to fit your data

Open the Terminal or Windows Command Prompt app on your computer.

Navigate to the Flywheel CLI.


Make sure to replace username with your username.

```
fw deid create ~/Documents/flywheel/deid_profile.yml
````
You will see the following message: Sample template successfully created

# Use CLI to run Gear

https://docs.flywheel.io/hc/en-us/articles/360061737113-Batch-Run-Gears

In [3]:
# Import packages
from getpass import getpass

import os
from pathlib import Path
import re
import time
import pprint
from datetime import datetime
from IPython.display import display, Image
import pandas as pd
# from permission import check_user_permission

In [4]:
project_name = 'K23'
task = 'rest'
subject_id = 'P18002'

fw = flywheel.Client()
project = fw.projects.find_one('label='+project_name)

mriqc_gear = fw.gears.find_first('gear.name=mriqc')           

# Download qa data

In [5]:
from pathlib import Path

# Path where the input files will be download
PATH_TO_DOWNLOAD = Path('/Users/TienTong/Documents/mriqc')
PATH_TO_DOWNLOAD.mkdir(exist_ok=True, parents=True)

In [ ]:
# Iterate over project sessions
for session in project.sessions.iter():
    session = session.reload()
    # Create a new directory for each session
    session_path = PATH_TO_DOWNLOAD / session.label
    # Here we will create the session directory if it does not exist in the path
    session_path.mkdir(exist_ok=True, parents=True)
    # Iterate over the acquisitions containers
    for acq in session.acquisitions.iter():
        for file_obj in acq.files:
            # We only want QA 
            if file_obj.type == 'qa':
                # Download the file to dest_path with the same file name on FW
                dest_path = session_path / file_obj.name
                # Here we will download the file if it is not there.
                if not dest_path.exists():
                    file_obj.download(dest_path)
     

# Create Dataframe from qc data

Aggregate data from everybody across all scans, will take a while

In [6]:
# Create a new DataFrame
mriqc_df = pd.DataFrame()

# Path to directory you want to store the csv files
dest_path = PATH_TO_DOWNLOAD

# Iterate over project sessions
for session in project.sessions.iter():
    session = session.reload()
    # Iterate all the acquisitions container
    for acq in session.acquisitions.iter():
        acq = acq.reload()
        for file_obj in acq.files:
            # We only want QA 
            if file_obj.type == 'qa':
                tmp_dict = {} 
                tmp_dict['FileName'] = file_obj.name 
                for info, value in file_obj.info.items():
                    # This is used to check if the value is a dictionary and whether they are bids metadata
                    if isinstance(value, dict) and info == "bids_meta":
                        tmp_dict.update(value)
                    elif isinstance(value, dict) == False:
                        tmp_dict[info] = value
                # Appending all of the info into the DataFrame
                mriqc_df = mriqc_df.append(tmp_dict, ignore_index=True).drop("dataset", axis=1)
                
# Create a new path with desired CSV file name
dest_path = PATH_TO_DOWNLOAD / "mriqc_allsubjects_allscans.csv"

# Display the DataFrame
display(mriqc_df)

# Convert DF to CSV file and save to the path initialize earlier
mriqc_df.to_csv(dest_path, index=False)

,FileName,modality,subject_id,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,...,summary_fg_k,summary_fg_mad,summary_fg_mean,summary_fg_median,summary_fg_n,summary_fg_p05,summary_fg_p95,summary_fg_stdv,task_id,tsnr
0,sub-P18001_T1w_mriqc.qa.html,T1w,P18001,0.545212,2.443376,0.555233,412.031372,4.809792,4.861087,4.756387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-P18001_task-rest_run-02_bold_mriqc.qa.html,bold,P18001,NaN,NaN,0.506777,1057.673950,3.013423,3.024000,3.019019,...,0.321237,2298.126953,5615.909180,5244.086426,33148.0,2384.674902,10066.544287,2364.334473,rest,71.320396
2,sub-P18001_task-whyhow_echo-02_bold_mriqc.qa.html,bold,P18001,NaN,NaN,0.446662,1936.618652,2.750049,2.517254,2.990637,...,0.173575,2489.143311,6133.746582,5735.113281,29571.0,2721.687256,10833.145020,2524.414307,whyhow,62.225273
3,sub-P18001_task-rest_run-01_echo-03_bold_mriqc...,bold,P18001,NaN,NaN,0.430964,1255.661499,2.617824,2.718177,2.636995,...,0.715390,1739.389771,4437.581055,4160.216797,27834.0,1716.038525,8074.306812,1928.147339,rest,53.427971
4,sub-P18001_task-rest_run-02_echo-03_bold_mriqc...,bold,P18001,NaN,NaN,0.431238,1284.072144,2.617162,2.716897,2.637913,...,0.688647,1734.803955,4445.000977,4171.080078,27882.0,1694.660162,8079.458423,1931.503296,rest,49.778358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,sub-P21014_task-rest_echo-01_bold_mriqc.qa.html,bold,P21014,NaN,NaN,0.501167,1730.866333,2.962557,2.520565,3.036911,...,2.017105,2313.503906,7169.359375,6734.879883,29269.0,3687.817920,12508.080078,2750.240234,rest,98.034752
726,sub-P21014_task-rest_echo-02_bold_mriqc.qa.html,bold,P21014,NaN,NaN,0.434595,2027.203857,2.754438,2.435459,2.924772,...,1.618083,1704.785156,5128.213379,4791.989746,26688.0,2523.532361,8951.605322,1996.893188,rest,68.856911
727,sub-P21014_task-rest_echo-03_bold_mriqc.qa.html,bold,P21014,NaN,NaN,0.420633,1265.948853,2.469882,2.255616,2.666966,...,2.659396,1224.306152,3628.167480,3380.540039,24968.0,1613.317322,6401.949805,1496.978516,rest,47.374149
728,sub-P21014_task-rest_bold_mriqc.qa.html,bold,P21014,NaN,NaN,0.491975,1609.490479,2.900684,2.526308,3.036466,...,1.788808,1599.765137,4869.350098,4557.916504,28543.0,2437.978906,8459.324609,1876.566895,rest,72.467957


In [28]:

# # files.info.BIDS.Filename
# test_session = project.sessions()[0]
# test_session.acquisitions.find('files.info.BIDS.Task=' + 
#                                 task +
#                                 ',files.info.BIDS.Modality!=sbref' +
#                                 ',files.type=nifti')[0]


# test_session.acquisitions.find('files.info.BIDS.template=anat_file'+
#                                 ',files.type=nifti')[0]

{'analyses': None,
 'collections': None,
 'created': datetime.datetime(2021, 9, 2, 19, 38, 47, 615000, tzinfo=tzutc()),
 'files': [{'classification': {'Intent': ['Structural'], 'Measurement': ['T1']},
            'created': datetime.datetime(2021, 9, 2, 19, 38, 48, 312000, tzinfo=tzutc()),
            'deid_log_id': None,
            'file_id': None,
            'hash': 'v0-sha384-925f6ad2fcc0ad6737d1df9ed72b9077928fbc9a9a94fa26fd748bb2e63da744d9251c4e1dea49c96c89c854ff6b788f',
            'id': '8e698703-8d08-47e9-bc0f-d9b9464d4674',
            'info': {'BIDS': {'Acq': '',
                              'Ce': '',
                              'Filename': 'sub-P18001_T1w.nii.gz',
                              'Folder': 'anat',
                              'Mod': '',
                              'Modality': 'T1w',
                              'Path': 'sub-P18001/anat',
                              'Rec': '',
                              'Run': '',
                              'ign